## Imports

In [1]:
import gradio as gr # for user interface
import json # for retrieving json files from website to get price history
import urllib.parse # encoding skin names to so it can be fetched through url 
import matplotlib.pyplot as plt # visualization 
import ast  
import requests # for web scrapping
import seaborn as sns # also for visualization
import pandas as pd # data framing processed data from json so that it can be plotted
from bs4 import BeautifulSoup # for web scrapping 
import os 
import random

## All for the LS TM model
import gc
import time
import numpy as np
import xgboost as xgb
import tensorflow as tf
import keras.backend as K
import matplotlib.dates as mdates # date-handling utilities
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler # to normalize data within a range
from sklearn.metrics import mean_absolute_error, mean_squared_error # to be used to count MAE and RMSE


In [2]:
def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

set_seeds(42)

## Make Lists

- load_skins() = Load skins from skins.txt file that was webscrapped from csgoskins.gg. This works from loading the items with format like ('Five-SeveN | Copper Galaxy', True, False) where the first boolean value is the modifier for stattrak and the second for souvenir. We also add the weapon_conditions for each weapons through a nested for loop. 
- search_list() = Makes a list of tuples to ease the searching process. 
- load_cases() = Loads cases from cases.txt file that was also webscrapped from csgoskins.gg. The logic is the same as load_skins(|)

In [3]:
weapon_conditions = [
    "(Factory New)", "(Minimal Wear)", "(Field-Tested)", "(Well-Worn)", "(Battle-Scarred)"
]

wear = [
    "Factory New", "Minimal Wear", "Field-Tested", "Well-Worn", "Battle-Scarred"
]

def load_skins():
    with open("skins.txt", "r", encoding="utf-8") as file:
        skins = [ast.literal_eval(line.strip()) for line in file.readlines()]  

    processed_skins = []
    for skin, is_stattrak, is_souvenir in skins:
        processed_skins.append(skin)  

        if is_stattrak:
            processed_skins.append(f"StatTrak {skin}")

        if is_souvenir:
            processed_skins.append(f"Souvenir {skin}")

    with_wear = []
    for skin in processed_skins:
        for conditions in weapon_conditions:
            with_wear.append(f"{skin} {conditions}")

    return with_wear

def search_list():
    with open("skins.txt", "r", encoding="utf-8") as file:
        skins = [ast.literal_eval(line.strip()) for line in file.readlines()]  

    separated_data = []
    for name, val1, val2 in skins:
        parts = name.split(" | ")
        
        if len(parts) == 2:  
            weapon, skin = parts
        else:
            weapon, skin = name, "Unknown"  
        weapon = weapon.replace("★", "")
        separated_data.append((weapon.strip(), skin.strip(), val1, val2))

    search_processed_skins = []
    for skins in separated_data:
        for w in wear:
                search_processed_skins.append((skins[0], skins[1], "Base", w))

        if skins[2]:
            for w in wear:
                search_processed_skins.append((skins[0], skins[1], "StatTrak", w))
        elif skins[3]:
            for w in wear:
                search_processed_skins.append((skins[0], skins[1], "Souvenir", w))

    return search_processed_skins

def load_cases():
    with open("case.txt", "r", encoding="utf-8") as file:
        cases = [line.strip() for line in file.readlines()]
    return cases 

## Searching Functions and URL Translation

- find_skin(sr_format, url_format, wep, s, w, m)
    - sr_format: result value from the search_list()
    - url_format: result value from the load_skin()
    - wep: name of the weapon 
    - s: name of the skin 
    - w: wear of the skin 
    - m: modifier of the skin(stattrak/souvenir)
- find_skin(): Works by matching the values given through the parameters with the list of tuples made through search_list(). Matched through their indexes. 
- find_case(c_data, name): Applies a simpler logic where it takes the data loaded from load_cases() function which is used in the c_data parameter. The name is also given to the function to match it with existing data. 
- create_url(item): Creates the link to the site containing the price history of an item in the steam market. The json will be downloaded in another phase. 

In [4]:
def find_skin(sr_format, url_format, wep,s,w,m):
    input_weapon = wep.lower()
    input_skin = s.lower() 
    input_wear = w.lower()
    input_modifier = m.lower()
    i = 0
    for items in sr_format:
        weapon = items[0].lower()
        skin = items[1].lower()
        modifier = items[2].lower()
        wear = items[3].lower()
        #print(skin,modifier,wear)
        if skin == input_skin and modifier == input_modifier and wear == input_wear and weapon == input_weapon:               
            print(f"Item Found in index : {i}")
            print(weapon, modifier, skin, wear)
            return url_format[i]
        i = i + 1
    return None 

def find_case(c_data, name):
    for cases in c_data:
        if cases.lower() == name.lower():
            return cases 

def create_url(item):
    url_format_base = "http://steamcommunity.com/market/pricehistory/?appid=730&market_hash_name="
    formatted = url_format_base+urllib.parse.quote(item, safe="★")
    return formatted.replace("%E2%98%85", "★")

## Get JSON
- get_json(url, file_name): just like its name says, it takes in the url made from create_url and gets the json from that site. It will be saved as the provided name through file_name

this also has the necessary cookies required to fetch the data

Note: For this step, it is important to have a steam account because without a steam account, you cannot fetch the price history of the item. The cookies are the ones that allows us to fetch the price history data of an item. These cookies can change once in a while so it is possible that the cookies written right now does not work. To get the cookies, you need to inspect the page where steam is logged in at and go to applications and find the sessionid and steamLoginSecure values to ensure that this part can run. Use the temporary account and replace cookie values with the logged in account. Inspect it on https://steamcommunity.com/market/ and make sure you are logged in

Temporary Steam Account: 
- Username: dia_project_2025
- Pass: Project2025

In [5]:
headers = { 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Referer": "https://steamcommunity.com",
}
# if it doesn't work, try another cookie 
cookies = { # make sure to censor in github
    "sessionid": "8aef10bfacdf994fdcf987c4",
    "steamLoginSecure": "76561199837440278%7C%7CeyAidHlwIjogIkpXVCIsICJhbGciOiAiRWREU0EiIH0.eyAiaXNzIjogInI6MDAwNl8yNjBDRDBERl9FMDcwNCIsICJzdWIiOiAiNzY1NjExOTk4Mzc0NDAyNzgiLCAiYXVkIjogWyAid2ViOmNvbW11bml0eSIgXSwgImV4cCI6IDE3NDMyNjU0MjksICJuYmYiOiAxNzM0NTM3MzkwLCAiaWF0IjogMTc0MzE3NzM5MCwgImp0aSI6ICIwMDAxXzI2MENEMEVGX0JBOUE1IiwgIm9hdCI6IDE3NDMwODk3NTgsICJydF9leHAiOiAxNzYxMzg2NzgyLCAicGVyIjogMCwgImlwX3N1YmplY3QiOiAiMjAzLjIxNy4xMzEuMTcwIiwgImlwX2NvbmZpcm1lciI6ICIyMDMuMjE3LjEzMS4xNzAiIH0.JG4dJBMKifLXh1Ciy1ZXGNLc5pIrVIanqjMNWuC-DSoYav3KITO8rXURa33FII9TYlI9gMl80aqLSXXL2cXvDQ",
}

def get_json(url, file_name):
    try:
        response = requests.get(url, headers=headers, cookies=cookies)
        if response.status_code == 400:
            print(f"400 Bad Request for {url}: {response.text}") 
        response.raise_for_status()
        data = response.json()

        filename = f"{file_name}.json"  
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4)

        print(f"Saved JSON as {filename} in the root directory.")
        return True  
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return False  

## Item Image
- This function is quite simple, you put in the name of the file and go to the listing page and then scrape the image from the site, download it to the same folder and then display it with gradio

In [6]:
def fetch_item_image(item, filename="item.jpg"):
    base_link = "https://steamcommunity.com/market/listings/730/"
    formatted = base_link + urllib.parse.quote(item, safe="★").replace("%E2%98%85", "★")

    print(formatted)

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
    }

    response = requests.get(formatted, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    image_div = soup.find("div", class_="market_listing_largeimage")
    if not image_div:
        return "Div not found."

    img_tag = image_div.find("img", {"src": True})
    if not img_tag:
        return "Image not found."

    image_url = img_tag["src"]
    img_response = requests.get(image_url, headers=headers)

    if img_response.status_code == 200:
        os.makedirs("images", exist_ok=True)
        
        file_path = os.path.join("images", filename)
        with open(file_path, "wb") as file:
            file.write(img_response.content)
        
        return file_path  
    else:
        return "Failed to download image."

## Create Function to be used for Prediction
- preprocess_data(file_path): Remove duplicates dates, taking the average price, and set proper date index.
- create_sequences(): Prepares time series data for machine learning models, particularly for sequential models like LSTMs.

In [7]:
def preprocess_data(file_path):
    
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file) # Load the file
    
    df = pd.DataFrame(data['prices'], columns=['Date', 'Price', 'Volume']) # Turns the json file into a dataframe
    df['Price'] = df['Price'].astype(float) # Make sure that price is a float type
    df['Volume'] = df['Volume'].astype(int) # Make sure that volume is a int type
    df['Date'] = pd.to_datetime(df['Date'], format='%b %d %Y %H: +0', errors='coerce') # Make sures that the date is formated correctly
    
    # Group by date, averaging Price & summing Volume
    df_grouped = df.groupby(df['Date'].dt.date).agg({'Price': 'mean', 'Volume': 'sum'}).reset_index()
    df_grouped.rename(columns={'Date': 'Date', 'Price': 'Price', 'Volume': 'Volume'}, inplace=True)
    
    # Convert Date column to datetime and set as index
    df_grouped['Date'] = pd.to_datetime(df_grouped['Date'])
    df_grouped.set_index('Date', inplace=True)
    
    return df_grouped

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # Input features (Price & Volume)
        y.append(data[i+seq_length][0])  # Predicting only Price
    return np.array(X), np.array(y)

## Price Prediction | ARIMA
- Predict prices using ARIMA model and plot actual vs predicted values.

In [8]:
def predict_price_arima(name):
    try:
        # To keep track of the time
        start_time = time.time()

        # Load and preprocess data
        df = preprocess_data('item.json')

        # Ensure the datetime index has a proper frequency
        df = df.asfreq('D').interpolate()

        # Scale both Price and Volume
        scaler = MinMaxScaler(feature_range=(0, 1))
        df[['Scaled_Price', 'Scaled_Volume']] = scaler.fit_transform(df[['Price', 'Volume']])

        # Split into training and test sets
        train_size = len(df) - 30  # Use all but last 30 days for training
        train_price, test_price = df['Scaled_Price'].iloc[:train_size], df['Scaled_Price'].iloc[train_size:]
        train_volume, test_volume = df['Scaled_Volume'].iloc[:train_size], df['Scaled_Volume'].iloc[train_size:]

        # Fit ARIMA model with Volume as exogenous variable
        model = ARIMA(train_price, order=(5,1,0), exog=train_volume)
        model_fit = model.fit()

        # Forecast the next 30 days with exogenous volume data
        predictions = model_fit.forecast(steps=30, exog=test_volume)

        # Reverse scaling for price predictions
        predictions = scaler.inverse_transform(np.column_stack((predictions, np.zeros_like(predictions))))[:, 0]

        # Get actual prices
        actual_prices = df['Price'].iloc[train_size:].values
        future_dates = df.index[train_size:]

        # Calculate evaluation metrics
        mae = mean_absolute_error(actual_prices, predictions)
        rmse = np.sqrt(mean_squared_error(actual_prices, predictions))
        mape = np.mean(np.abs((actual_prices - predictions) / actual_prices)) * 100

        print(f"MAE ARIMA: {mae:.4f}")
        print(f"MAPE ARIMA: {mape:.4f}%")
        print(f"RMSE ARIMA: {rmse:.4f}")

        # # Calculate price difference
        # last_actual_price = df['Price'].iloc[train_size - 1]  # Last actual price before prediction
        # last_predicted_price = predictions[-1, 0]  # Last predicted price after 30 days
        # price_change = last_predicted_price - last_actual_price
        # percentage_change = (price_change / last_actual_price) * 100

        # print(f"Price Change: {price_change:.2f}")
        # print(f"Percentage Change: {percentage_change:.2f}%")

        # Plot actual vs predicted prices
        plt.figure(figsize=(12, 6))
        plt.plot(df.index[-60:], df['Price'][-60:], label='Actual Prices', marker='o')
        plt.plot(future_dates, predictions, label='Predicted Prices (ARIMA)', linestyle='dashed', color='red', marker='o')
        plt.legend()
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.title(f'ARIMA Backtesting (Actual vs Predicted - 30 Days) for {name}')

        # Save the plot
        predict_path_arima = "images/prediction_arima.png"
        plt.savefig(predict_path_arima)
        plt.close()

        # Calculate execution time
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Execution Time: {execution_time:.4f} seconds \n")

        return predict_path_arima

    except Exception as e:
        print(f"Error: {e}")

## Price Prediction | LSTM
- Predict prices using LSTM model and plot actual vs predicted values

In [9]:
def predict_price_lstm(name):
    try:
        # Clear previous TensorFlow sessions
        K.clear_session()
        gc.collect()
        tf.compat.v1.reset_default_graph()

        # To keep track of execution time
        start_time = time.time()

        # Load and preprocess data
        df = preprocess_data('item.json')

        # Ensure the datetime index has a proper frequency
        df = df.asfreq('D').interpolate()

        # Scale both Price and Volume
        scaler = MinMaxScaler(feature_range=(0, 1))
        df[['Scaled_Price', 'Scaled_Volume']] = scaler.fit_transform(df[['Price', 'Volume']])

        # Split into training and test sets
        seq_length = 30
        train_size = len(df) - seq_length
        train_data = df[['Scaled_Price', 'Scaled_Volume']].iloc[:train_size].values
        test_data = df[['Scaled_Price', 'Scaled_Volume']].iloc[train_size:].values

        X_train, y_train = create_sequences(train_data, seq_length)
        X_test, y_test = create_sequences(np.concatenate([train_data[-seq_length:], test_data]), seq_length)

        # Reshape input for LSTM (Samples, Time Steps, Features)
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 2))
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 2))

        # Define LSTM model
        model = Sequential([
            Input(shape=(seq_length, 2)),  
            LSTM(50, return_sequences=True),
            Dropout(0.2),
            LSTM(50, return_sequences=False),
            Dropout(0.2),
            Dense(25, activation='relu'),
            Dense(1)
        ])

        model.compile(optimizer='adam', loss='mean_squared_error')

        # Use EarlyStopping to prevent overfitting
        early_stopping = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)

        # Train the model
        model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stopping]) 

        # Make predictions
        predictions = model.predict(X_test)
        predictions = scaler.inverse_transform(np.column_stack((predictions, np.zeros_like(predictions))))[:, 0]

        # Get actual prices
        actual_prices = df['Price'].iloc[train_size:].values
        future_dates = df.index[train_size:]

        # Calculate evaluation metrics
        mae = mean_absolute_error(actual_prices, predictions)
        rmse = np.sqrt(mean_squared_error(actual_prices, predictions))
        mape = np.mean(np.abs((actual_prices - predictions) / actual_prices)) * 100

        print(f"MAE LSTM: {mae:.4f}")
        print(f"MAPE LSTM: {mape:.4f}%")
        print(f"RMSE LSTM: {rmse:.4f}")

        # Plot actual vs predicted prices
        plt.figure(figsize=(12, 6))
        plt.plot(df.index[-60:], df['Price'][-60:], label='Actual Prices', marker='o')
        plt.plot(future_dates, predictions, label='Predicted Prices (LSTM)', linestyle='dashed', color='red', marker='o')
        plt.legend()
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.title(f'LSTM Backtesting (Actual vs Predicted - 30 Days) for {name}')

        # Save the plot
        predict_path = "images/prediction_lstm.png"
        plt.savefig(predict_path)
        plt.close()

        # Calculate execution time
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Execution Time: {execution_time:.4f} seconds \n")

        return predict_path

    except Exception as e:
        print(f"Error: {e}")

## Price Prediction | XGBoost
- Predict prices using XGBoost model and plot actual vs predicted values.

In [10]:
def predict_price_xgboost(name):
    try:
        # To keep track of the time
        start_time = time.time()

        # Load and preprocess data
        df = preprocess_data('item.json')

        # Ensure the datetime index has a proper frequency
        df = df.asfreq('D').interpolate()

        # Scale both Price and Volume
        scaler = MinMaxScaler(feature_range=(0, 1))
        df[['Scaled_Price', 'Scaled_Volume']] = scaler.fit_transform(df[['Price', 'Volume']])

        # Split into training and test sets
        seq_length = 30
        train_size = len(df) - seq_length 
        train_data = df[['Scaled_Price', 'Scaled_Volume']].iloc[:train_size].values
        test_data = df[['Scaled_Price', 'Scaled_Volume']].iloc[train_size:].values

        # Prepare sequences
        X_train, y_train = create_sequences(train_data, seq_length)
        X_test, y_test = create_sequences(np.concatenate([train_data[-seq_length:], test_data]), seq_length)

        # Flatten input for XGBoost
        X_train = X_train.reshape(X_train.shape[0], -1)  # 2D shape for XGBoost
        X_test = X_test.reshape(X_test.shape[0], -1)

        # Define XGBoost model
        model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.5, max_depth=5)
        model.fit(X_train, y_train)

        # Make predictions
        predictions = model.predict(X_test)

        # Reverse scaling for price predictions
        predictions = scaler.inverse_transform(np.column_stack((predictions, np.zeros_like(predictions))))[:, 0]

        # Get actual prices
        actual_prices = df['Price'].iloc[train_size:].values
        future_dates = df.index[train_size:]

        # Calculate evaluation metrics
        mae = mean_absolute_error(actual_prices, predictions)
        rmse = np.sqrt(mean_squared_error(actual_prices, predictions))
        mape = np.mean(np.abs((actual_prices - predictions) / actual_prices)) * 100

        print(f"MAE XGBoost: {mae:.4f}")
        print(f"MAPE XGBoost: {mape:.4f}%")
        print(f"RMSE XGBoost: {rmse:.4f}")

        # # Calculate price difference
        # last_actual_price = df['Price'].iloc[train_size - 1]  # Last actual price before prediction
        # last_predicted_price = predictions[-1, 0]  # Last predicted price after 30 days
        # price_change = last_predicted_price - last_actual_price
        # percentage_change = (price_change / last_actual_price) * 100

        # print(f"Price Change: {price_change:.2f}")
        # print(f"Percentage Change: {percentage_change:.2f}%")

        # Plot actual vs predicted prices
        plt.figure(figsize=(12, 6))
        plt.plot(df.index[-60:], df['Price'][-60:], label='Actual Prices', marker='o')
        plt.plot(future_dates, predictions, label='Predicted Prices (XGBoost)', linestyle='dashed', color='red', marker='o')
        plt.legend()
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.title(f'XGBoost Backtesting (Actual vs Predicted - 30 Days) for {name}')

        # Save the plot
        predict_path = "images/prediction_xgboost.png"
        plt.savefig(predict_path)
        plt.close()

        # Calculate execution time
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Execution Time: {execution_time:.4f} seconds \n")

        return predict_path

    except Exception as e:
        print(f"Error: {e}")


## Gradio Interface

All of the functions and how they work are implemented into two functions which are:
- process_skin_input(): Takes in user inputs and process them 
- process_case_input(): Does the same thing, just for cases 

This then is used in conjunction with the gradio interface to make it easier for users to interact with the agent.

In [ ]:
def process_skin_input(weapon_input, skin_name, condition, modifiers):
    url_format = load_skins()  # Load skin list
    sr_format = search_list()  # Load search list

    search = find_skin(sr_format, url_format, weapon_input, skin_name, condition, modifiers)
    
    if not search:
        return "Skin not found. Please check your input.", None, None, None
    
    img = fetch_item_image(search)

    url = create_url(search)
    get_json(url, "item")  

    file_path = "item.json"
    with open(file_path, "r") as file:
        data = json.load(file)

    df = pd.DataFrame(data["prices"], columns=["timestamp", "price", "volume"])
    df["price"] = df["price"].astype(float)
    df["volume"] = df["volume"].astype(int)
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%b %d %Y %H: +0", errors="coerce")
    df = df.dropna().sort_values("timestamp")

    sns.set_style("darkgrid")
    plt.figure(figsize=(12, 6))
    sns.lineplot(x=df["timestamp"], y=df["price"], color="b", label="Price")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.title(f"Price Over Time for {search}")
    plt.xticks(rotation=45)
    plt.legend()

    temp_path = "images/temp_chart.png"
    plt.savefig(temp_path)
    plt.close()

    return search, img, temp_path, predict_price_arima(search), predict_price_lstm(search), predict_price_xgboost(search)

def process_case_input(case_name):
    case_data = load_cases()
    search = find_case(case_data, case_name)

    if not search:
        return "Case not found. Please check your input.", None, None, None

    img = fetch_item_image(search)

    url = create_url(search)
    get_json(url, "item")  

    file_path = "item.json"
    with open(file_path, "r") as file:
        data = json.load(file)

    df = pd.DataFrame(data["prices"], columns=["timestamp", "price", "volume"])
    df["price"] = df["price"].astype(float)
    df["volume"] = df["volume"].astype(int)
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%b %d %Y %H: +0", errors="coerce")
    df = df.dropna().sort_values("timestamp")

    sns.set_style("darkgrid")
    plt.figure(figsize=(12, 6))
    sns.lineplot(x=df["timestamp"], y=df["price"], color="b", label="Price")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.title(f"Price Over Time for {search}")
    plt.xticks(rotation=45)
    plt.legend()

    temp_path = "images/temp_chart.png"
    plt.savefig(temp_path)
    plt.close()

    return search, img, temp_path, predict_price_arima(search), predict_price_lstm(search), predict_price_xgboost(search)

with gr.Blocks() as demo:
    gr.Markdown("# Counter-Strike Market Analyzer")
    selection = gr.Radio(["Skins", "Case", "Knifes and Gloves"], label="Select Item Type [Skins/Case]", value=None)

    with gr.Column(visible=False) as skins_interface: 
        with gr.Row():
            weapon_input = gr.Dropdown(
                    ['AK-47', 'AUG', 'AWP', 'CZ75-Auto', 'Desert Eagle', 'Dual Berettas', 'FAMAS', 'Five-SeveN', 
                    'G3SG1', 'Galil AR', 'Glock-18', 'M249', 'M4A1-S', 'M4A4', 'MAG-7', 'MAC-10', 'MP5-SD', 'MP9', 
                    'Negev', 'Nova', 'P2000', 'P250', 'P90', 'PP-Bizon', 'R8 Revolver', 'SCAR-20', 'SG 553', 
                    'SSG 08', 'Sawed-Off', 'Tec-9', 'UMP-45', 'USP-S', 'XM1014', 'Zeus x27'],
                    label="Select a Weapon"
                )
            skin_name = gr.Textbox(label="Enter Skin Name: ", placeholder="e.g, Dragon Lore")
        with gr.Row():
            condition = gr.Dropdown(["Factory New", "Minimal Wear", "Field Tested", "Well Worn", "Battle-Scarred"], label="Condition")
            modifiers = gr.Dropdown([ "Base", "StatTrak", "Souvenir"], label="Pick the Modifiers")
        btn1 = gr.Button("Submit")
       
        text_msg = gr.Textbox(label="Result", interactive=False)
        item_image = gr.Image(label="Item Image")
        with gr.Row():
            priceChart = gr.Image(label="Price Trend")
            predictChartARIMA = gr.Image(label="Price Prediction ARIMA")
            predictChartLSTM = gr.Image(label="Price Prediction LSTM")
            predictChartXGB = gr.Image(label="Price Prediction XGB")


        btn1.click(
            fn=process_skin_input,
            inputs=[weapon_input, skin_name, condition, modifiers],
            outputs=[text_msg, item_image, priceChart, predictChartARIMA, predictChartLSTM, predictChartXGB]
        )

    with gr.Column(visible=False) as cases_interface:  
        case_input = gr.Dropdown(
                    [
                    "Gallery Case", "Kilowatt Case", "Revolution Case", "Recoil Case", "Dreams & Nightmares Case",
                    "Chroma 2 Case", "Chroma 3 Case", "Chroma Case", "Clutch Case", "CS20 Case", "CS:GO Weapon Case", 
                    "CS:GO Weapon Case 2", "CS:GO Weapon Case 3", "Danger Zone Case", "eSports 2013 Case", 
                    "eSports 2013 Winter Case", "eSports 2014 Summer Case", "Falchion Case", "Fracture Case", "Gamma 2 Case", 
                    "Gamma Case", "Glove Case", "Horizon Case", "Huntsman Weapon Case", "Operation Bravo Case", "Operation Breakout Weapon Case",
                    "Operation Broken Fang Case", "Operation Hydra Case", "Operation Phoenix Weapon Case", "Operation Riptide Case",
                    "Operation Vanguard Weapon Case", "Operation Wildfire Case", "Prisma 2 Case", "Prisma Case", "Revolver Case",
                    "Shadow Case", "Shattered Web Case", "Snakebite Case", "Spectrum 2 Case", "Spectrum Case", "Winter Offensive Weapon Case"
                    ],
                    label="Select a Case"
                )
        btn2 = gr.Button("Submit")
       
        text_msg = gr.Textbox(label="Result", interactive=False)
        item_image = gr.Image(label="Item Image")
        with gr.Row():
            priceChart = gr.Image(label="Price Trend")
            predictChartARIMA = gr.Image(label="Price Prediction ARIMA")
            predictChartLSTM = gr.Image(label="Price Prediction LSTM")
            predictChartXGB = gr.Image(label="Price Prediction XGB")

        btn2.click(
            fn=process_case_input,
            inputs=[case_input],
            outputs=[text_msg, item_image, priceChart, predictChartARIMA, predictChartLSTM, predictChartXGB]
        )

    with gr.Column(visible=False) as kg_interface:  
        with gr.Row():
            kg_input = gr.Dropdown(
                    [
                    "Bayonet", "Bowie Knife", "Butterfly Knife", "Classic Knife",
                    "Falchion Knife", "Flip Knife", "Gut Knife", "Huntsman Knife",
                    "Karambit", "Kukri Knife", "M9 Bayonet", "Navaja Knife", "Nomad Knife",
                    "Paracord Knife", "Shadow Daggers", "Skeleton Knife", "Stiletto Knife",
                    "Survival Knife", "Talon Knife", "Ursus Knife",
                    "Bloodhound Gloves", "Broken Fang Gloves", "Driver Gloves", "Hand Wraps",
                    "Hydra Gloves", "Moto Gloves", "Specialist Gloves", "Sport Gloves"
                    ],
                    label="Select a Weapon"
                )
            skin_name = gr.Textbox(label="Enter Skin Name: ", placeholder="e.g, Dragon Lore")
        with gr.Row():
            condition = gr.Dropdown(["Factory New", "Minimal Wear", "Field Tested", "Well Worn", "Battle-Scarred"], label="Condition")
            modifiers = gr.Dropdown(["Base","StatTrak"], label="Pick the Modifiers")
        btn3 = gr.Button("Submit")
       
        text_msg = gr.Textbox(label="Result", interactive=False)
        item_image = gr.Image(label="Item Image")
        with gr.Row():
            priceChart = gr.Image(label="Price Trend")
            predictChartARIMA = gr.Image(label="Price Prediction ARIMA")
            predictChartLSTM = gr.Image(label="Price Prediction LSTM")
            predictChartXGB = gr.Image(label="Price Prediction XGB")

        btn3.click(
            fn=process_skin_input,
            inputs=[kg_input, skin_name, condition, modifiers],
            outputs=[text_msg, item_image, predictChartARIMA, predictChartLSTM, predictChartXGB]
        )


    def select_type(selection):
        if selection == "Skins":
            return gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)
        elif selection == "Case":
            return gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)
        elif selection == "Knifes and Gloves":
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=True)
        else:
            return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)


    selection.change(
        fn=select_type,
        inputs=[selection],
        outputs=[skins_interface, cases_interface, kg_interface]
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


https://steamcommunity.com/market/listings/730/Shadow%20Case
Saved JSON as item.json in the root directory.


c:\Users\vagan\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


MAE ARIMA: 0.0938
MAPE ARIMA: 4.5221%
RMSE ARIMA: 0.1195
Execution Time: 13.6724 seconds 


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 617ms/step
MAE LSTM: 0.1799
MAPE LSTM: 8.9260%
RMSE LSTM: 0.1860
Execution Time: 169.9281 seconds 

MAE XGBoost: 0.2134
MAPE XGBoost: 10.3562%
RMSE XGBoost: 0.2421
Execution Time: 1.0973 seconds 

https://steamcommunity.com/market/listings/730/Shattered%20Web%20Case
Saved JSON as item.json in the root directory.


c:\Users\vagan\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


...

MAE ARIMA: 0.0444
MAPE ARIMA: 11.5069%
RMSE ARIMA: 0.0500
Execution Time: 6.9045 seconds 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
MAE LSTM: 0.0110
MAPE LSTM: 2.7999%
RMSE LSTM: 0.0131
Execution Time: 72.4893 seconds 

MAE XGBoost: 0.0083
MAPE XGBoost: 2.0631%
RMSE XGBoost: 0.0106
Execution Time: 0.3726 seconds 

https://steamcommunity.com/market/listings/730/Spectrum%202%20Case
Saved JSON as item.json in the root directory.


c:\Users\vagan\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


MAE ARIMA: 0.1004
MAPE ARIMA: 2.8441%
RMSE ARIMA: 0.1187
Execution Time: 8.4104 seconds 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
MAE LSTM: 0.4466
MAPE LSTM: 12.4757%
RMSE LSTM: 0.4503
Execution Time: 154.8743 seconds 

MAE XGBoost: 0.1056
MAPE XGBoost: 2.9870%
RMSE XGBoost: 0.1254
Execution Time: 0.4088 seconds 

https://steamcommunity.com/market/listings/730/Spectrum%20Case
Saved JSON as item.json in the root directory.


c:\Users\vagan\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


MAE ARIMA: 0.1482
MAPE ARIMA: 2.7939%
RMSE ARIMA: 0.1827
Execution Time: 7.4094 seconds 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step
MAE LSTM: 0.5844
MAPE LSTM: 11.1603%
RMSE LSTM: 0.5904
Execution Time: 168.1339 seconds 

MAE XGBoost: 0.1109
MAPE XGBoost: 2.1106%
RMSE XGBoost: 0.1315
Execution Time: 0.6658 seconds 

https://steamcommunity.com/market/listings/730/Winter%20Offensive%20Weapon%20Case
Saved JSON as item.json in the root directory.
MAE ARIMA: 0.3726
MAPE ARIMA: 3.9831%
RMSE ARIMA: 0.4386
Execution Time: 4.6583 seconds 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
MAE LSTM: 0.4212
MAPE LSTM: 4.5199%
RMSE LSTM: 0.4875
Execution Time: 244.8098 seconds 

MAE XGBoost: 0.2154
MAPE XGBoost: 2.3294%
RMSE XGBoost: 0.2736
Execution Time: 0.7970 seconds 

